# Block Aligner Benchmark Analysis and Visualizations

This notebook contains code for collecting, cleaning, and analyzing data produced by block aligner's experiments.

To run this, you will need to install all the libraries imported below, along with [altair-saver](https://github.com/altair-viz/altair_saver) and [altair-data-server](https://github.com/altair-viz/altair_data_server), which has some extra dependencies for PDF saving.

Run each cell one by one to reproduce the experiments. This may take a while. For accurate benchmarking, it is recommended to run the entire notebook in the command line with `nbconvert`.

In [1]:
import altair as alt
from altair_saver import save
from altair import datum
import pandas as pd
from io import StringIO

alt.data_transformers.enable("data_server")

DataTransformerRegistry.enable('data_server')

In [2]:
def csv_to_pandas(csv, d = "\\s*,\\s*", t = None):
    s = StringIO("\n".join(csv))
    data = pd.read_csv(s, sep = d, thousands = t, comment = "#", engine = "python")
    return data

## Prefix Scan Benchmark

In [3]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- prefix_scan | grep 'bench:' | awk '{print $2"\t"$5}'
output.insert(0, "algorithm\ttime")
output

['algorithm\ttime', 'bench_naive_prefix_scan\t26', 'bench_opt_prefix_scan\t18']

In [4]:
data = csv_to_pandas(output, d = "\t", t = ",")
data

,algorithm,time
0,bench_naive_prefix_scan,26
1,bench_opt_prefix_scan,18


In [5]:
data["algorithm"] = data["algorithm"].map({
    "bench_naive_prefix_scan": "naive",
    "bench_opt_prefix_scan": "ours"
})
data

,algorithm,time
0,naive,26
1,ours,18


Prefix Scan Benchmark (AVX2)

In [6]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("time", axis = alt.Axis(title = "time (ns)")),
    y = "algorithm",
    color = alt.Color("algorithm", legend = None)
).properties(
    width = 150
)
save(c, "prefix_scan_bench.pdf")
c

alt.Chart(...)

## Random Data Benchmark

In [7]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- bench_ | grep 'bench:' | grep -v 'prefix_scan' | awk '{print $2"\t"$5}'
output

['bench_parasailors_aa_1000_10000\t52,435,895',
 'bench_parasailors_aa_100_1000\t608,499',
 'bench_parasailors_aa_10_100\t20,155',
 'bench_rustbio_aa_100_1000\t15,363,409',
 'bench_rustbio_aa_10_100\t170,271',
 'bench_scan_aa_1000_10000\t231,190',
 'bench_scan_aa_1000_10000_insert\t4,913,574',
 'bench_scan_aa_1000_10000_small\t220,464',
 'bench_scan_aa_1000_10000_trace\t1,861,712',
 'bench_scan_aa_100_1000\t26,275',
 'bench_scan_aa_100_1000_insert\t50,462',
 'bench_scan_aa_100_1000_small\t23,579',
 'bench_scan_aa_100_1000_trace\t514,997',
 'bench_scan_aa_10_100\t3,939',
 'bench_scan_aa_10_100_insert\t4,071',
 'bench_scan_aa_10_100_small\t3,486',
 'bench_scan_aa_10_100_trace\t368,967',
 'bench_scan_nuc_1000_10000\t223,067',
 'bench_scan_nuc_100_1000\t24,061',
 'bench_triple_accel_1000_10000\t8,404,724',
 'bench_triple_accel_100_1000\t24,589']

In [8]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["parasailors_aa", "rustbio_aa", "scan_aa", "scan_nuc", "triple_accel"]
new_names = ["parasailors\tprotein", "rust bio\tprotein", "ours\tprotein", "ours\tnucleotide", "triple accel\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'parasailors\tprotein\t1000\t10000\tdefault\t52,435,895',
 'parasailors\tprotein\t100\t1000\tdefault\t608,499',
 'parasailors\tprotein\t10\t100\tdefault\t20,155',
 'rust bio\tprotein\t100\t1000\tdefault\t15,363,409',
 'rust bio\tprotein\t10\t100\tdefault\t170,271',
 'ours\tprotein\t1000\t10000\tdefault\t231,190',
 'ours\tprotein\t1000\t10000\tinsert\t4,913,574',
 'ours\tprotein\t1000\t10000\tsmall\t220,464',
 'ours\tprotein\t1000\t10000\ttrace\t1,861,712',
 'ours\tprotein\t100\t1000\tdefault\t26,275',
 'ours\tprotein\t100\t1000\tinsert\t50,462',
 'ours\tprotein\t100\t1000\tsmall\t23,579',
 'ours\tprotein\t100\t1000\ttrace\t514,997',
 'ours\tprotein\t10\t100\tdefault\t3,939',
 'ours\tprotein\t10\t100\tinsert\t4,071',
 'ours\tprotein\t10\t100\tsmall\t3,486',
 'ours\tprotein\t10\t100\ttrace\t368,967',
 'ours\tnucleotide\t1000\t10000\tdefault\t223,067',
 'ours\tnucleotide\t100\t1000\tdefault\t24,061',
 'triple accel\tnucleotide\t1000\t100

In [9]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,parasailors,protein,1000,10000,default,52435895
1,parasailors,protein,100,1000,default,608499
2,parasailors,protein,10,100,default,20155
3,rust bio,protein,100,1000,default,15363409
4,rust bio,protein,10,100,default,170271
5,ours,protein,1000,10000,default,231190
6,ours,protein,1000,10000,insert,4913574
7,ours,protein,1000,10000,small,220464
8,ours,protein,1000,10000,trace,1861712
9,ours,protein,100,1000,default,26275


In [10]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (AVX2)

In [11]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench.pdf")
c

alt.Chart(...)

Random DNA Sequences Benchmark (AVX2)

In [12]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000])),
    shape = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000]))
).transform_filter(
    datum.alphabet == "nucleotide"
).properties(
    width = 200,
    height = 50
)
save(c, "random_dna_bench.pdf")
c

alt.Chart(...)

## Uniclust 30 Data Benchmark

In [13]:
output = !cd .. && cargo run --example uc_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, size, time',
 'ours (no trace), uc30, 32-32, 0.056515289',
 'ours (no trace), uc30 0.95, 32-32, 0.060549953',
 'ours (no trace), uc30, 32-256, 0.089667067',
 'ours (no trace), uc30 0.95, 32-256, 0.07712594',
 'ours (no trace), uc30, 256-256, 0.199199121',
 'ours (no trace), uc30 0.95, 256-256, 0.22163839',
 'ours (trace), uc30, 32-256, 0.168939332',
 'ours (trace), uc30 0.95, 32-256, 0.150488372',
 'parasail, uc30, full, 0.885421801',
 'parasail, uc30 0.95, full, 1.028783795']

In [14]:
data = csv_to_pandas(output)
data

,algorithm,dataset,size,time
0,ours (no trace),uc30,32-32,0.056515
1,ours (no trace),uc30 0.95,32-32,0.060550
2,ours (no trace),uc30,32-256,0.089667
3,ours (no trace),uc30 0.95,32-256,0.077126
4,ours (no trace),uc30,256-256,0.199199
5,ours (no trace),uc30 0.95,256-256,0.221638
6,ours (trace),uc30,32-256,0.168939
7,ours (trace),uc30 0.95,32-256,0.150488
8,parasail,uc30,full,0.885422
9,parasail,uc30 0.95,full,1.028784


Uniclust30 Benchmark (AVX2)

In [15]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("algorithm", axis = None),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = "algorithm"
).transform_filter(
    (datum.size == "32-256") | (datum.algorithm == "parasail")
)
t = c.mark_text(dy = -4, size = 8).encode(text = alt.Text("time", format = ".2f"), color = alt.value("black"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
).configure_range(
    category = {"scheme": "dark2"}
)
save(c, "uniclust30_bench.pdf")
c

alt.FacetChart(...)

Uniclust30 Block Size Benchmark (AVX2)

In [16]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", axis = None, sort = ["32-32", "32-256", "256-256"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = alt.Color("size", title = "block size", sort = ["32-32", "32-256", "256-256"])
).transform_filter(
    datum.algorithm == "ours (no trace)"
)
t = c.mark_text(dy = -4, size = 8).encode(text = alt.Text("time", format = ".2f"), color = alt.value("black"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
)
save(c, "uniclust30_size_bench.pdf")
c

alt.FacetChart(...)

## DNA Global Alignment Benchmark

In [17]:
output = !cd .. && cargo run --example nanopore_bench_global --release --features simd_avx2 --quiet
output

['# time (s)',
 'dataset, algorithm, time',
 'illumina, ours (1%-1%), 0.19954773599994785',
 'illumina, ours (1%-10%), 0.21103766700000928',
 'illumina, edlib, 0.3688567379999959',
 'illumina, ksw_extz2_sse (1%), 0.5450279550000086',
 'illumina, ksw_extz2_sse (10%), 0.5442524660000194',
 'illumina, wfa2, 0.07222540399997783',
 'illumina, wfa2 adaptive, 0.07643134800001018',
 'illumina, parasail, 1.905899847999988',
 'nanopore 1kbp, ours (1%-1%), 0.20423522500000035',
 'nanopore 1kbp, ours (1%-10%), 0.2545604999999995',
 'nanopore 1kbp, edlib, 0.361402042999999',
 'nanopore 1kbp, ksw_extz2_sse (1%), 0.5809077849999988',
 'nanopore 1kbp, ksw_extz2_sse (10%), 1.2340126780000036',
 'nanopore 1kbp, wfa2, 1.182402372000001',
 'nanopore 1kbp, wfa2 adaptive, 0.9811043799999979',
 'nanopore 1kbp, parasail, 3.922164622000011',
 'nanopore <10kbp, ours (1%-1%), 0.7581100459999974',
 'nanopore <10kbp, ours (1%-10%), 1.082943893999997',
 'nanopore <10kbp, edlib, 1.2777344540000006',
 'nanopore <10kb

In [18]:
data = csv_to_pandas(output)
data

,dataset,algorithm,time
0,illumina,ours (1%-1%),0.199548
1,illumina,ours (1%-10%),0.211038
2,illumina,edlib,0.368857
3,illumina,ksw_extz2_sse (1%),0.545028
4,illumina,ksw_extz2_sse (10%),0.544252
5,illumina,wfa2,0.072225
6,illumina,wfa2 adaptive,0.076431
7,illumina,parasail,1.905900
8,nanopore 1kbp,ours (1%-1%),0.204235
9,nanopore 1kbp,ours (1%-10%),0.254560


DNA Global Alignment Benchmark (AVX2)

In [19]:
algos = ["ours (1%-1%)", "ours (1%-10%)", "edlib", "ksw_extz2_sse (1%)", "ksw_extz2_sse (10%)", "wfa2", "wfa2 adaptive", "parasail"]
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("algorithm", sort = algos, title = None),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)")),
    color = alt.Color("algorithm", legend = None)
)
t = c.mark_text(dy = -4, size = 8).encode(text = alt.Text("time", format = ".2f"), color = alt.value("black"))
c = (c + t).properties(
    width = 140,
    height = 140
).facet(
    facet = alt.Facet("dataset", title = None, header = alt.Header(orient = "top")),
    columns = 2
).resolve_scale(
    y = "independent"
).configure_axisY(
    labelPadding = 18,
    labelAlign = "left"
)
save(c, "dna_global_bench.pdf")
c

alt.FacetChart(...)

## Nanopore Data Benchmark Setup

To run the benchmarks below, you need to clone the following repos, place them in the same directory where this repo (block aligner) is located, and follow their setup instructions:
* [diff-bench-paper](https://github.com/Daniel-Liu-c0deb0t/diff-bench-paper)
* [adaptivebandbench](https://github.com/Daniel-Liu-c0deb0t/adaptivebandbench)

## Nanopore Data Benchmark

In [20]:
output = !cd .. && cargo run --example nanopore_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, time',
 'ours (no trace 32-32), nanopore 25kbp, 1.033478184',
 'ours (no trace 32-32), random, 2.550499957',
 'ours (trace 32-32), nanopore 25kbp, 1.412760824',
 'ours (trace 32-32), random, 3.400294701',
 'ours (trace 32-64), nanopore 25kbp, 1.6694597629999999',
 'ours (trace 32-64), random, 3.5342896550000003']

In [21]:
data = csv_to_pandas(output)
data

,algorithm,dataset,time
0,ours (no trace 32-32),nanopore 25kbp,1.033478
1,ours (no trace 32-32),random,2.550500
2,ours (trace 32-32),nanopore 25kbp,1.412761
3,ours (trace 32-32),random,3.400295
4,ours (trace 32-64),nanopore 25kbp,1.669460
5,ours (trace 32-64),random,3.534290


In [22]:
output2 = !cd ../../diff-bench-paper/supplementary_data/benchmark_codes && ./custom_bench.sh

for i, o in enumerate(output2):
    if o.startswith("cells("):
        break
output2 = output2[i + 1:]

output2.insert(0, "algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail")
output2

['algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail',
 'editdist\t476567000\t170629000\t67690000\t714886000\t6880489\t0',
 'non-diff\t681717000\t270098000\t61707000\t1013522000\t27124786\t52',
 'diff-raw\t634119000\t213761000\t64883000\t912763000\t27291141\t32',
 'libgaba\t452611000\t157111000\t32888000\t642610000\t27121546\t53',
 'edlib\t28046347000\t19390997000\t106224000\t47543568000\t37\t0',
 'seqan\t90098655000\t0\t0\t90098655000\t0\t0']

In [23]:
data2 = csv_to_pandas(output2, d = "\t")
data2

,algorithm,fill time,trace time,convert time,total time,score,fail
0,editdist,476567000,170629000,67690000,714886000,6880489,0
1,non-diff,681717000,270098000,61707000,1013522000,27124786,52
2,diff-raw,634119000,213761000,64883000,912763000,27291141,32
3,libgaba,452611000,157111000,32888000,642610000,27121546,53
4,edlib,28046347000,19390997000,106224000,47543568000,37,0
5,seqan,90098655000,0,0,90098655000,0,0


In [24]:
cleaned2 = data2.drop(columns = ["trace time", "convert time", "total time", "score", "fail"])
cleaned2 = cleaned2.rename(columns = {"fill time": "time"})
cleaned2["time"] /= 1e9
cleaned2

,algorithm,time
0,editdist,0.476567
1,non-diff,0.681717
2,diff-raw,0.634119
3,libgaba,0.452611
4,edlib,28.046347
5,seqan,90.098655


In [25]:
cleaned = data.drop(index = [1, 3, 5])
cleaned = cleaned.drop(columns = ["dataset"])
cleaned = cleaned.append(cleaned2, ignore_index = True)
cleaned

,algorithm,time
0,ours (no trace 32-32),1.033478
1,ours (trace 32-32),1.412761
2,ours (trace 32-64),1.669460
3,editdist,0.476567
4,non-diff,0.681717
5,diff-raw,0.634119
6,libgaba,0.452611
7,edlib,28.046347
8,seqan,90.098655


25kbp Nanopore Reads Benchmark (AVX2)

In [26]:
chart1 = alt.Chart(cleaned).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm != "ours (trace 32-32)") & (datum.algorithm != "ours (no trace 32-32)") & (datum.algorithm != "ours (trace 32-64)"))

chart2 = alt.Chart(cleaned).mark_point(color = "red", filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm == "ours (trace 32-32)") | (datum.algorithm == "ours (no trace 32-32)") | (datum.algorithm == "ours (trace 32-64)"))

c = (chart1 + chart2).properties(
    width = 150,
    height = 150
)
save(c, "nanopore_bench.pdf")
c

alt.LayerChart(...)

## Sequence-to-Profile Alignment Benchmark

In [27]:
output = !cd .. && cargo run --example pssm_bench --release --features simd_avx2 --quiet
output

['size, time',
 '32-32, 0.150358618',
 '32-64, 0.174074908',
 '32-128, 0.200178733',
 '128-128, 0.202823522',
 'parasail, 0.579455554',
 '# Done!']

In [28]:
data = csv_to_pandas(output)
data

,size,time
0,32-32,0.150359
1,32-64,0.174075
2,32-128,0.200179
3,128-128,0.202824
4,parasail,0.579456


In [29]:
table = data.copy()
table = table.rename(columns = {"size": "block size"})
table["time"] = table["time"].map("{:.2}".format)
print(table)

  block size  time
0      32-32  0.15
1      32-64  0.17
2     32-128   0.2
3    128-128   0.2
4   parasail  0.58


SCOP Sequence-to-Profile Alignment Benchmark (AVX2)

In [30]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", title = "block size", sort = ["32-32", "32-64", "32-128", "128-128", "parasail"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)")),
    color = alt.Color("size", sort = ["32-32", "32-64", "32-128", "128-128", "parasail"], legend = None)
).transform_filter(
    datum.size != "2048-2048"
).properties(
    width = 75,
    height = 100
)
t = c.mark_text(dy = -4, size = 8).encode(text = alt.Text("time", format = ".2f"), color = alt.value("black"))
c = c + t
save(c, "pssm_size_bench.pdf")
c

alt.LayerChart(...)

## WASM SIMD

[Wasmtime](https://wasmtime.dev/) is needed to run the webassembly code.

In [31]:
output = !CARGO_TARGET_WASM32_WASI_RUNNER="wasmtime --wasm-features simd --" cargo bench --target=wasm32-wasi --features simd_wasm --quiet -- --nocapture | grep 'bench:' | awk '{print $2"\t"$5}'
output

['bench_rustbio_aa_100_1000\t24,067,946',
 'bench_rustbio_aa_10_100\t254,862',
 'bench_scan_aa_1000_10000\t829,802',
 'bench_scan_aa_1000_10000_insert\t10,785,694',
 'bench_scan_aa_1000_10000_small\t578,308',
 'bench_scan_aa_1000_10000_trace\t1,995,010',
 'bench_scan_aa_100_1000\t74,559',
 'bench_scan_aa_100_1000_insert\t156,641',
 'bench_scan_aa_100_1000_small\t57,182',
 'bench_scan_aa_100_1000_trace\t220,585',
 'bench_scan_aa_10_100\t6,824',
 'bench_scan_aa_10_100_insert\t7,029',
 'bench_scan_aa_10_100_small\t5,345',
 'bench_scan_aa_10_100_trace\t84,180',
 'bench_scan_nuc_1000_10000\t556,340',
 'bench_scan_nuc_100_1000\t54,636']

In [32]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["rustbio_aa", "scan_aa", "scan_nuc"]
new_names = ["rust bio\tprotein", "ours\tprotein", "ours\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'rust bio\tprotein\t100\t1000\tdefault\t24,067,946',
 'rust bio\tprotein\t10\t100\tdefault\t254,862',
 'ours\tprotein\t1000\t10000\tdefault\t829,802',
 'ours\tprotein\t1000\t10000\tinsert\t10,785,694',
 'ours\tprotein\t1000\t10000\tsmall\t578,308',
 'ours\tprotein\t1000\t10000\ttrace\t1,995,010',
 'ours\tprotein\t100\t1000\tdefault\t74,559',
 'ours\tprotein\t100\t1000\tinsert\t156,641',
 'ours\tprotein\t100\t1000\tsmall\t57,182',
 'ours\tprotein\t100\t1000\ttrace\t220,585',
 'ours\tprotein\t10\t100\tdefault\t6,824',
 'ours\tprotein\t10\t100\tinsert\t7,029',
 'ours\tprotein\t10\t100\tsmall\t5,345',
 'ours\tprotein\t10\t100\ttrace\t84,180',
 'ours\tnucleotide\t1000\t10000\tdefault\t556,340',
 'ours\tnucleotide\t100\t1000\tdefault\t54,636']

In [33]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,rust bio,protein,100,1000,default,24067946
1,rust bio,protein,10,100,default,254862
2,ours,protein,1000,10000,default,829802
3,ours,protein,1000,10000,insert,10785694
4,ours,protein,1000,10000,small,578308
5,ours,protein,1000,10000,trace,1995010
6,ours,protein,100,1000,default,74559
7,ours,protein,100,1000,insert,156641
8,ours,protein,100,1000,small,57182
9,ours,protein,100,1000,trace,220585


In [34]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (WASM SIMD)

In [35]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench_wasm.pdf")
c

alt.Chart(...)